In [1]:
# This notebook will create the datasets for the analysis

import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# Set the random seed for reproducibility
np.random.seed(50)

In [2]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta

def generate_campaign_performance(n_samples):
    np.random.seed(50)

    # Datas
    end_date = datetime.today()
    start_date = end_date - timedelta(days=30)
    date = [start_date + timedelta(days=np.random.randint(0, 31)) for _ in range(n_samples)]

    # Channel and campaign type
    channels = ['Meta', 'Google', 'TikTok', 'Affiliate', 'Taboola', 'Moloco']
    channel_probs = [0.2, 0.5, 0.06, 0.1, 0.04, 0.1]
    channel = np.random.choice(channels, size=n_samples, p=channel_probs)
    campaign_type = np.where(channel == 'Moloco', 'retargeting', 'acquisition')

    # Campaign ID
    campaign_ids_dict = {ch: [f'{ch.upper()}_CAMP_{i:03d}' for i in range(1, 51)] for ch in channels}
    campaign_id = [np.random.choice(campaign_ids_dict[ch]) for ch in channel]

    # Device an OS
    device_type = np.random.choice(['Mobile', 'Desktop'], size=n_samples, p=[0.7, 0.3])
    os = np.full(n_samples, 'Desktop')
    mobile_mask = device_type == 'Mobile'
    os[mobile_mask] = np.random.choice(['Android', 'iOS'], size=mobile_mask.sum(), p=[0.6, 0.4])

    # Countries
    country = np.random.choice(['USA', 'UK', 'Canada', 'Germany', 'India', 'Brazil', 'Japan'], size=n_samples)

    # Impressions
    impressions = np.random.exponential(scale=8000, size=n_samples).astype(int)
    impressions = np.clip(impressions, 1000, 20000)

    # Clicks via CTR
    ctr = np.random.uniform(0.005, 0.05, size=n_samples)
    clicks = (impressions * ctr).astype(int)
    clicks = np.clip(clicks, 1, None)

    # Conversions via CVR
    cvr = np.random.uniform(0.02, 0.30, size=n_samples)
    conversions = (clicks * cvr).astype(int)
    conversions = np.clip(conversions, 0, None)

    # Spend and Revenue
    spend = np.round(np.random.exponential(scale=5000, size=n_samples), 2)
    spend = np.clip(spend, 1, 15000)
    rev_per_conv = np.round(np.random.uniform(20, 100, size=n_samples), 2)
    revenue = np.round(conversions * rev_per_conv, 2)

    # DataFrame
    df = pd.DataFrame({
        'campaign_id': campaign_id,
        'date': date,
        'channel': channel,
        'campaign_type': campaign_type,
        'device_type': device_type,
        'os': os,
        'country': country,
        'impressions': impressions,
        'clicks': clicks,
        'conversions': conversions,
        'spend_usd': spend,
        'revenue_usd': revenue
    })

    return df

# Dataset with 10K rows
n_samples = 10000
campaign_data = generate_campaign_performance(n_samples)
campaign_data.head()


,campaign_id,date,channel,campaign_type,device_type,os,country,impressions,clicks,conversions,spend_usd,revenue_usd
0,GOOGLE_CAMP_037,2025-06-28 13:50:10.134732,Google,acquisition,Mobile,iOS,India,1000,20,4,2476.93,119.36
1,GOOGLE_CAMP_007,2025-06-12 13:50:10.134732,Google,acquisition,Mobile,iOS,India,2170,42,7,8598.35,506.59
2,MOLOCO_CAMP_022,2025-06-23 13:50:10.134732,Moloco,retargeting,Mobile,Android,USA,1000,29,2,311.88,65.92
3,TIKTOK_CAMP_034,2025-06-25 13:50:10.134732,TikTok,acquisition,Mobile,Android,Japan,2032,11,2,7720.85,103.96
4,GOOGLE_CAMP_031,2025-06-13 13:50:10.134732,Google,acquisition,Desktop,Desktop,Germany,4802,51,1,9043.12,73.00


In [3]:
revenue_by_channel = campaign_data.groupby('channel')['revenue_usd'].sum().reset_index()
revenue_by_channel = revenue_by_channel.sort_values(by='revenue_usd', ascending=False)
print(revenue_by_channel)


     channel  revenue_usd
1     Google   9545414.40
2       Meta   3508049.12
3     Moloco   1919883.76
0  Affiliate   1825205.64
5     TikTok   1159378.26
4    Taboola    871276.72


In [ ]:
#Change campaign_data Date to date format
campaign_data['date'] = pd.to_datetime(campaign_data['date']).dt.date

In [15]:
def generate_user_cohort(n_users):
    np.random.seed(50)

    # Totoal revenue per channel
    channel_revenue = {
        'Google': 9427702.02,
        'Meta': 3739916.71,
        'Affiliate': 1903039.09,
        'Moloco': 1848481.78,
        'TikTok': 1087645.11,
        'Taboola': 907329.17
    }

    channels = list(channel_revenue.keys())
    channel_probs = [0.5, 0.2, 0.1, 0.1, 0.06, 0.04]  

    # IDs and dates
    user_id = [f'u_{i:05d}' for i in range(n_users)]
    end_date = datetime.today()
    start_date = end_date - timedelta(days=30)
    signup_date = [start_date + timedelta(days=np.random.randint(0, 31)) for _ in range(n_users)]

    # Channel and campaign_ids
    channel = np.random.choice(channels, size=n_users, p=channel_probs)
    campaign_ids_dict = {ch: [f'{ch.upper()}_CAMP_{i:03d}' for i in range(1, 51)] for ch in channels}
    campaign_id = [np.random.choice(campaign_ids_dict[ch]) for ch in channel]
    campaign_type = np.where(channel == 'Moloco', 'retargeting', 'acquisition')

    # Device and OS
    device_type = np.random.choice(['Mobile', 'Desktop'], size=n_users, p=[0.7, 0.3])
    os = np.full(n_users, 'Desktop')
    mobile_mask = device_type == 'Mobile'
    os[mobile_mask] = np.random.choice(['Android', 'iOS'], size=mobile_mask.sum(), p=[0.6, 0.4])

    # Countries
    country = np.random.choice(['USA', 'UK', 'Canada', 'Germany', 'India', 'Brazil', 'Japan'], size=n_users)

    # Conversions: 60%
    converted_mask = np.random.rand(n_users) < 0.6
    converted_date = [signup_date[i] + timedelta(days=np.random.randint(0, 15)) if converted_mask[i] else pd.NaT for i in range(n_users)]
    conversion_lag_days = [(converted_date[i] - signup_date[i]).days if converted_mask[i] else np.nan for i in range(n_users)]

    # Revenue per user, proportional to channel
    revenue_usd = np.zeros(n_users)
    for ch in channels:
        idx = np.where((channel == ch) & converted_mask)[0]
        n = len(idx)
        if n > 0:
            base = np.random.exponential(scale=100, size=n)
            base = base / base.sum() * channel_revenue[ch]
            revenue_usd[idx] = np.round(base, 2)

    # DataFrame
    user_cohort = pd.DataFrame({
        'user_id': user_id,
        'campaign_id': campaign_id,
        'signup_date': signup_date,
        'converted_date': converted_date,
        'conversion_lag_days': conversion_lag_days,
        'channel': channel,
        'campaign_type': campaign_type,
        'device_type': device_type,
        'os': os,
        'country': country,
        'revenue_usd': revenue_usd
    })

    return user_cohort

# Conversions from the campaign dataset sum 315134, since we are considering that 60% of the signup actually converted 
n_users = 525223
user_cohort = generate_user_cohort(n_users)


user_cohort.head()


,user_id,campaign_id,signup_date,converted_date,conversion_lag_days,channel,campaign_type,device_type,os,country,revenue_usd
0,u_00000,AFFILIATE_CAMP_038,2025-06-28 14:08:09.444727,NaT,NaN,Affiliate,acquisition,Mobile,Android,Germany,0.00
1,u_00001,GOOGLE_CAMP_042,2025-06-12 14:08:09.444727,2025-06-24 14:08:09.444727,12.0,Google,acquisition,Mobile,Android,Canada,94.71
2,u_00002,MOLOCO_CAMP_044,2025-06-23 14:08:09.444727,NaT,NaN,Moloco,retargeting,Mobile,iOS,USA,0.00
3,u_00003,AFFILIATE_CAMP_005,2025-06-25 14:08:09.444727,2025-07-06 14:08:09.444727,11.0,Affiliate,acquisition,Mobile,iOS,USA,42.79
4,u_00004,GOOGLE_CAMP_037,2025-06-13 14:08:09.444727,2025-06-24 14:08:09.444727,11.0,Google,acquisition,Mobile,Android,USA,194.65


In [16]:
# Since not everyone converted, I'll add a new Boolean column based on conversion or not
user_cohort['user_Converted'] = user_cohort['converted_date'].notna()

In [17]:
user_cohort['user_Converted'] = user_cohort['user_Converted'].astype(bool)

In [18]:
user_cohort['signup_date'] = user_cohort['signup_date'].dt.date
user_cohort['converted_date'] = user_cohort['converted_date'].dt.date

In [19]:
user_cohort.to_csv('user_cohort.csv', index=False, sep=",", quoting=1)

In [20]:
user_cohort.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 525223 entries, 0 to 525222
Data columns (total 12 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   user_id              525223 non-null  object 
 1   campaign_id          525223 non-null  object 
 2   signup_date          525223 non-null  object 
 3   converted_date       315219 non-null  object 
 4   conversion_lag_days  315219 non-null  float64
 5   channel              525223 non-null  object 
 6   campaign_type        525223 non-null  object 
 7   device_type          525223 non-null  object 
 8   os                   525223 non-null  object 
 9   country              525223 non-null  object 
 10  revenue_usd          525223 non-null  float64
 11  user_Converted       525223 non-null  bool   
dtypes: bool(1), float64(2), object(9)
memory usage: 44.6+ MB


In [5]:
import numpy as np
import pandas as pd

def generate_ad_creatives(n_creatives=200):
    np.random.seed(50)

    # Channels and proportions
    channels = ['Meta', 'Google', 'TikTok', 'Affiliate', 'Taboola', 'Moloco']
    channel_probs = [0.2, 0.5, 0.06, 0.1, 0.04, 0.1]

    # Sizes
    formats = ['video', 'static', 'carousel']
    format_probs = [0.5, 0.4, 0.1]
    image_styles = ['lifestyle', 'product', 'food', 'minimalist']
    languages = ['en', 'de', 'fr', 'pt', 'es']

    # IDs and channels
    creative_id = [f'CR{i:04d}' for i in range(n_creatives)]
    channel = np.random.choice(channels, size=n_creatives, p=channel_probs)

    # Campaign id per channels
    campaign_ids_dict = {ch: [f'{ch.upper()}_CAMP_{i:03d}' for i in range(1, 51)] for ch in channels}
    campaign_id = [np.random.choice(campaign_ids_dict[ch]) for ch in channel]

    # Format, style and language
    creative_format = np.random.choice(formats, size=n_creatives, p=format_probs)
    image_style = np.random.choice(image_styles, size=n_creatives)
    language = np.random.choice(languages, size=n_creatives)


    # A/B
    is_experiment = np.random.choice([True, False], size=n_creatives, p=[0.3, 0.7])
    variant = [np.random.choice(['A', 'B', 'C']) if exp else None for exp in is_experiment]

    # DataFrame 
    ad_creatives = pd.DataFrame({
        'creative_id': creative_id,
        'campaign_id': campaign_id,
        'channel': channel,
        'format': creative_format,
        'image_style': image_style,
        'is_experiment': is_experiment,
        'variant': variant,
        'language': language
    })

    return ad_creatives

# Gerar os dados
ad_creatives = generate_ad_creatives()
ad_creatives.head()


,creative_id,campaign_id,channel,format,image_style,is_experiment,variant,language
0,CR0000,GOOGLE_CAMP_019,Google,static,lifestyle,False,None,es
1,CR0001,GOOGLE_CAMP_045,Google,static,food,False,None,es
2,CR0002,GOOGLE_CAMP_038,Google,video,product,True,B,fr
3,CR0003,GOOGLE_CAMP_011,Google,static,lifestyle,False,None,es
4,CR0004,GOOGLE_CAMP_038,Google,video,product,False,None,fr


In [11]:
ad_creatives.to_csv('ad_creatives.csv', index=False, sep=",", quoting=1)
user_cohort.to_csv('user_cohort.csv', index=False, sep=",", quoting=1)
campaign_data.to_csv('campaign_data.csv', index=False, sep=",", quoting=1)